In [1]:
import importlib
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
importlib.reload(network_tools)
importlib.reload(firewall_tools)

<module 'tools.firewall_tools' from '/home/c0ff3k1ll3r/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/firewall_tools.py'>

In [2]:
alerts = await network_tools.get_fast_alerts(time_window=5)
alerts

{'security_events': {'alerts': [],
  'count': 0,
  'time_window_minutes': 5,
  'timestamp': '2025-08-11T11:43:26.155351'}}

In [3]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': 'd4f49be72012',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-08-10T14:36:51.70819098Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.3'},
  {'id': '23803120ebc8',
   'name': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'status': 'running',
   'created': '2025-08-10T14:36:51.395661281Z',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip_address': '172.20.0.2'}]}

In [4]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [69]:
await firewall_tools.remove_firewall_rule([1])

INFO:tools.firewall_tools:Removing firewall rules: [1]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.2']}

In [5]:
honeypot_ip = '172.20.0.3' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.3 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.3 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.3 -> 192.168.100.2/24 (tcp)'}